In [1]:
import os
from os import listdir
from os.path import isfile, join

import pandas as pd
from itertools import islice
import numpy as np

from skimage.external import tifffile
from skimage.external.tifffile import imread

import matplotlib.pyplot as plt

import ipywidgets as widgets

import random


from scipy.ndimage.morphology import binary_erosion as br
from skimage import morphology as skmor

from scipy import ndimage
from PIL import Image, ImageDraw

import numpy.ma as ma

from skimage.measure import label, regionprops

## to calculate
1.	nucleus_DAPI_total
2.	outer_DAPI_total
3.	inner_DAPI_total
4.	nucleus_DAPI_hetChrom
5.	outer_DAPI_hetChrom
6.	inner_DAPI_hetChrom
7.	nucleus_DAPI_euChrom
8.	outer_DAPI_euChrom
9.	inner_DAPI_euChrom



# gathering files
deleting the columns with the signals to recalculate and collect data

# 20200207_new data

In [2]:
bigDir=r'Z:\CookLab\Liu\20190816_organizedData_MCM_loading\20200203_new data'

In [3]:
file3=f'{bigDir}\\cellinfo_200203_v10_10.csv'
data3=pd.read_csv(file3)

In [4]:
file4=f'{bigDir}\\cellinfo_200203_v10_50.csv'
data4=pd.read_csv(file4)

# h3k9me3 3rd dataset

In [12]:
bigDir=r'Z:\CookLab\Liu\20190816_organizedData_MCM_loading\20191202_new dataset'

In [13]:
file5=f'{bigDir}\\signals_new_v6_10.csv'
data5=pd.read_csv(file5)

In [14]:
file6=f'{bigDir}\\signals_new_v6_50.csv'
data6=pd.read_csv(file6)

# 20200217_geminin data

In [27]:
bigDir=r'Z:\CookLab\Liu\20190816_organizedData_MCM_loading\20200217_geminin'

In [28]:
file=f'{bigDir}\\200217_cellinfo_v9_10.csv'
data=pd.read_csv(file)

In [29]:
file2=f'{bigDir}\\200217_cellinfo_v9_50.csv'
data2=pd.read_csv(file2)

# DAPI calculations

In [5]:
def calculate_DAPI_signals (data,percentage):
    # progress bar
    i=0
    progBar=widgets.IntProgress(
        value=i,
        min=0,
        max=len(data),
        step=1,
        description='Progress:',
        orientation='horizontal'
    )
    display(progBar)
    percentageCpy= percentage
    
    for i,myCell in islice(data.iterrows(), i,None):
        '''
        if myCell.group=='CTL':
            percentage = 20
        '''
        ## opening cell image
        try:
            myImage=imread(myCell.path) 
        except:
            myCell.path=myCell.path.replace('.tif','.tiff')
            myImage=imread(myCell.path)
        
        ## channel for DAPI calculations is 2 for both datasets
        myChannel=myImage[:,2,:,:]
        
        

        ## setting arrays for names 
        
        ### for column names
        pixels=['total','het','eu']
        components=['nuc','inner','outer']
        
        ### for masks names
        dirs=['segmentation_nucleus_Otsu','erosion_nuc_mask','erosion_nuc_mask',
              f'segmentation_{percentage}',f'segmentation_{percentage}_erosion',f'segmentation_{percentage}_erosion',
             f'segmentation_{percentage}_eu',f'segmentation_{percentage}_eu_erosion',f'segmentation_{percentage}_eu_erosion',]

        files=['_nucleus.tif','_eroded_9^5_inner_mask_20.tif','_eroded_9^5_outer_mask_20.tif',
                   '_hetChrom.tif', '_hetChrom_inner.tif', '_hetChrom_outer.tif',
                   '_euChrom.tif', '_euChrom_inner.tif','_euChrom_outer.tif']

        df=pd.DataFrame(columns=['nuc_vol_total','inner_vol_total','outer_vol_total',
                                 'nuc_mcm_total','inner_mcm_total','outer_mcm_total',
                                'nuc_vol_het','inner_vol_het','outer_vol_het',
                                 'nuc_mcm_het','inner_mcm_het','outer_mcm_het',
                                 'nuc_vol_eu','inner_vol_eu','outer_vol_eu',
                                'nuc_mcm_eu','inner_mcm_eu', 'outer_mcm_eu'])

        ## calculating signals
        for k in range(len(pixels)):
            for j in range(len(components)):
                ## copying image every time new signal is calculated
                myIm=myImage.copy()
                myChannel=myIm[:,2,:,:]
                
                ## column name set up based on the component and the pixels
                currColName=f'{components[j]}_DAPI_{pixels[k]}'

                ## opening mask images
                ### the indexing is in such way so that the image opened matches up with the name being used for calculation
                direc=myCell.path.replace('data_tiff',dirs[(j)+(k*3)])
                try:
                    direc=direc.replace('.tif',files[(k*3)+(j)])
                    mask=imread(direc)  
                except:
                    direc=direc.replace('_nucleus.tiff',files[(k*3)+(j)])
                    mask=imread(direc)

                ## making the mask as binary
                mask=mask.astype(bool)
                
                ## masking the image with the mask
                myChannel[~mask]=0
                
                ## calculating signals
                df[f'{currColName}']=myChannel.sum(-1).sum(-1)
        
        ## saving signals into original/big dataframe
        for myMeasurement in df.columns:
            myString=f"data.loc[i,'{myMeasurement}']=np.sum(df.{myMeasurement})"
            exec(myString)
            
        percentage = percentageCpy
        
        progBar.value=progBar.value+1

In [6]:
## calculating signals for 200207 dataset
calculate_DAPI_signals(data3,10)
data3.to_csv(file3.replace("v10", "v11"))
calculate_DAPI_signals(data4,50)
data4.to_csv(file4.replace("v10", "v11"))

IntProgress(value=0, description='Progress:', max=146)

IntProgress(value=0, description='Progress:', max=146)

In [22]:
## calculating signals for 191202 dataset
calculate_DAPI_signals(data5,10)
data5.to_csv(file5.replace("v6", "v7"))
calculate_DAPI_signals(data6,50)
data6.to_csv(file6.replace("v6", "v7"))

IntProgress(value=0, description='Progress:', max=276)

IntProgress(value=0, description='Progress:', max=276)

In [30]:
## calculating signals for 200217 dataset
calculate_DAPI_signals(data,10)
data.to_csv(file.replace("v9", "v10"))
calculate_DAPI_signals(data2,50)
data2.to_csv(file2.replace("v9", "v10"))

IntProgress(value=0, description='Progress:', max=145)

IntProgress(value=0, description='Progress:', max=145)